# Subword Tokenizer
- 기계에게 아무리 많은 단어를 학습시켜도, 세상의 모든 단어를 알려줄 수는 없는 노릇입니다. 만약, 기계가 모르는 단어가 등장하면 그 단어를 단어 집합에 없는 단어란 의미에서 OOV(Out-Of-Vocabulary) 또는 UNK(Unknown Token)라고 표현합니다. 기계가 문제를 풀 때, 모르는 단어가 등장하면 (사람도 마찬가지지만) 주어진 문제를 푸는 것이 까다로워 집니다. 이와 같이 모르는 단어로 인해 문제를 푸는 것이 까다로워지는 상황을 OOV 문제라고 합니다.
- 서브워드 분리(Subword segmenation) 작업은 하나의 단어는 더 작은 단위의 의미있는 여러 서브워드들(Ex) birthplace = birth + place)의 조합으로 구성된 경우가 많기 때문에, 하나의 단어를 여러 서브워드로 분리해서 단어를 인코딩 및 임베딩하겠다는 의도를 가진 전처리 작업입니다. 이를 통해 OOV나 희귀 단어, 신조어와 같은 문제를 완화시킬 수 있습니다. 실제로 언어의 특성에 따라 영어권 언어나 한국어는 서브워드 분리를 시도했을 때 어느정도 의미있는 단위로 나누는 것이 가능합니다. 이 책에서는 이런 작업을 하는 토크나이저를 서브워드 토크나이저라고 명명하겠습니다.
- 이번 챕터에서는 OOV 문제를 완화하는 대표적인 서브워드 분리 알고리즘인 BPE(Byte Pair Encoding) 알고리즘을 소개하고, 다음 챕터에서 이 알고리즘을 실무에서 사용할 수 있도록 구현한 센텐스피스(Sentencepiece)를 소개합니다.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import seaborn as sb
import pandas as pd
import sentencepiece as sp
import csv

## 2. Subword Tokenizer

### 2-1. IMDB

In [ ]:
# 디렉토리 안에 모든 파일들을 DataFrame 형태로 읽어옵니다.
# 구체적으로 문장(sentence)과 문장의 감정상태의 확신정도(sentiment=1~10)를 읽어옵니다.
def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(directory):
        with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

# 긍정(postive) 예제와 부정(negative) 예제를 하나의 dataframe으로 합치고 
# 긍정 혹은 부정을 나타내는 polarity 컬럼을 추가하고 데이터를 랜덤하게 섞습니다.
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos"))
    neg_df = load_directory_data(os.path.join(directory, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# IMDB 영화 리뷰 데이터셋을 다운받고 전처리를 진행합니다.
def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(fname="aclImdb.tar.gz", origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", extract=True)

    train_df = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "test"))
  
    return train_df, test_df

In [ ]:
wget.download("http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz")

100% [........................................................................] 84125825 / 84125825

'aclImdb_v1.tar.gz'

In [ ]:
import gzip

gb_file = gzip.open("aclImdb_v1.tar.gz", 'rb')

In [ ]:
gb_file

<gzip _io.BufferedReader name='aclImdb_v1.tar.gz' 0x2be045f4648>

In [ ]:
zipfile.ZipFile("aclImdb_v1.tar.gz").extractall()

BadZipFile: File is not a zip file

In [ ]:
train_df, test_df = download_and_load_datasets()

KeyboardInterrupt: 

In [ ]:
train_df

In [ ]:
train_df["sentence"]

0        This movie was awful. The ending was absolutel...
1        Patrick Channing (Jeff Kober) is a disciple of...
2        The first two-thirds of this biopic of fetish ...
3        Fortunately for us Real McCoy fans (most likel...
4        I wouldn't call it awful, but nothing at all s...
                               ...                        
24995    I loved the Batman tv series and was really lo...
24996    One of the best comedians ever. I've seen this...
24997    One thing I always liked about Robert Ludlum t...
24998    Just Cause takes some of the best parts of thr...
24999    Everything in this film is bad , the story , t...
Name: sentence, Length: 25000, dtype: object

In [ ]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    train_df["sentence"], target_vocab_size=2**13)

In [ ]:
print(tokenizer.subwords)

NameError: name 'tokenizer' is not defined

In [ ]:
print(train_df["sentence"][20])

NameError: name 'train_df' is not defined

In [ ]:
print("Tokenized sample question: {}".format(tokenizer.encode(train_df["sentence"][20])))

NameError: name 'tokenizer' is not defined

In [ ]:
# train_df에 존재하는 문장 중 일부를 발췌
sample_string = "It"s mind-blowing to me that this film was even made."

# encode
tokenized_string = tokenizer.encode(sample_string)
print ("정수 인코딩 후의 문장 {}".format(tokenized_string))

# encoding한 문장을 다시 decode
original_string = tokenizer.decode(tokenized_string)
print ("기존 문장: {}".format(original_string))

assert original_string == sample_string

정수 인코딩 후의 문장 [135, 7968, 8, 965, 7974, 2405, 34, 7, 105, 13, 14, 32, 18, 78, 677, 7975]
기존 문장: It's mind-blowing to me that this film was even made.


In [ ]:
print("단어 집합의 크기(Vocab size) :", tokenizer.vocab_size)

단어 집합의 크기(Vocab size) : 8185


In [ ]:
for ts in tokenized_string:
  print ("{} ----> {}".format(ts, tokenizer.decode([ts])))

135 ----> It
7968 ----> '
8 ----> s 
965 ----> mind
7974 ----> -
2405 ----> blow
34 ----> ing 
7 ----> to 
105 ----> me 
13 ----> that 
14 ----> this 
32 ----> film 
18 ----> was 
78 ----> even 
677 ----> made
7975 ----> .


In [ ]:
# 앞서 실습한 문장에 even 뒤에 임의로 xyz 추가
sample_string = "It"s mind-blowing to me that this film was evenxyz made."

# encode
tokenized_string = tokenizer.encode(sample_string)
print ("정수 인코딩 후의 문장 {}".format(tokenized_string))

# encoding한 문장을 다시 decode
original_string = tokenizer.decode(tokenized_string)
print ("기존 문장: {}".format(original_string))

assert original_string == sample_string

정수 인코딩 후의 문장 [135, 7968, 8, 965, 7974, 2405, 34, 7, 105, 13, 14, 32, 18, 6373, 8049, 8050, 990, 677, 7975]
기존 문장: It's mind-blowing to me that this film was evenxyz made.


In [ ]:
for ts in tokenized_string:
  print ("{} ----> {}".format(ts, tokenizer.decode([ts])))

135 ----> It
7968 ----> '
8 ----> s 
965 ----> mind
7974 ----> -
2405 ----> blow
34 ----> ing 
7 ----> to 
105 ----> me 
13 ----> that 
14 ----> this 
32 ----> film 
18 ----> was 
6373 ----> even
8049 ----> x
8050 ----> y
990 ----> z 
677 ----> made
7975 ----> .


## 2. tfds.deprecated.text.SubwordTextEncoder
- SubwordTextEncoder는 텐서플로우를 통해 사용할 수 있는 서브워드 토크나이저입니다. BPE와 유사한 알고리즘인 Wordpiece Model을 채택하였으며, 패키지를 통해 쉽게 단어들을 서브워드들로 분리할 수 있습니다. SubwordTextEncoder를 통해서 IMDB 영화 리뷰 데이터와 네이버 영화 리뷰 데이터에 대해서 토큰화 작업을 수행해봅시다.

### 2-2. 네이버 영화 리뷰

In [ ]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7fe80cc83390>)

In [ ]:
train_data = pd.read_table("ratings_train.txt")
test_data = pd.read_table("ratings_test.txt")

train_data = train_data.dropna(subset=["document"])

In [ ]:
train_data.head()

id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1

In [ ]:
tkn = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(train_data["document"], target_vocab_size=2**13)

In [ ]:
print(f"len(tkn.subwords) : {len(tkn.subwords)}")
print(tkn.subwords[:100])

len(tkn.subwords) : 7997
['. ', '..', '영화', '이_', '...', '의_', '는_', '도_', '다', ', ', '을_', '고_', '은_', '가_', '에_', '.. ', '한_', '너무_', '정말_', '를_', '고', '게_', '영화_', '지', '... ', '진짜_', '이', '다_', '요', '만_', '? ', '과_', '나', '가', '서_', '지_', '로_', '으로_', '아', '어', '....', '음', '한', '수_', '와_', '도', '네', '그냥_', '나_', '더_', '왜_', '이런_', '면_', '기', '하고_', '보고_', '하는_', '서', '좀_', '리', '자', '스', '안', '! ', '에서_', '영화를_', '미', 'ㅋㅋ', '네요', '시', '주', '라', '는', '오', '없는_', '에', '해', '사', '!!', '영화는_', '마', '잘_', '수', '영화가_', '만', '본_', '로', '그_', '지만_', '대', '은', '비', '의', '일', '개', '있는_', '없다', '함', '구', '하']


In [ ]:
print(train_data["document"][20])
print(tkn.encode(train_data["document"][20]))
for subw in tkn.encode(train_data["document"][20]):
    print(f"{subw:5d} ----> {tkn.decode([subw])}")

나름 심오한 뜻도 있는 듯. 그냥 학생이 선생과 놀아나는 영화는 절대 아님
[669, 4700, 17, 1749, 8, 96, 131, 1, 48, 2239, 4, 7466, 32, 1274, 2655, 7, 80, 749, 1254]
  669 ----> 나름 
 4700 ----> 심오
   17 ----> 한 
 1749 ----> 뜻
    8 ----> 도 
   96 ----> 있는 
  131 ----> 듯
    1 ----> . 
   48 ----> 그냥 
 2239 ----> 학생
    4 ----> 이 
 7466 ----> 선생
   32 ----> 과 
 1274 ----> 놀
 2655 ----> 아나
    7 ----> 는 
   80 ----> 영화는 
  749 ----> 절대 
 1254 ----> 아님


In [ ]:
print("이 영화 굉장히 재밌다 킄핫핫ㅎ")
print(tkn.encode("이 영화 굉장히 재밌다 킄핫핫ㅎ"))
for subw in tkn.encode("이 영화 굉장히 재밌다 킄핫핫ㅎ"):
    print(f"{subw:5d} ----> {tkn.decode([subw])}")

이 영화 굉장히 재밌다 킄핫핫ㅎ
[4, 23, 1364, 2157, 8235, 8128, 8130, 8235, 8147, 8169, 8235, 8147, 8169, 393]
    4 ----> 이 
   23 ----> 영화 
 1364 ----> 굉장히 
 2157 ----> 재밌다 
 8235 ----> �
 8128 ----> �
 8130 ----> �
 8235 ----> �
 8147 ----> �
 8169 ----> �
 8235 ----> �
 8147 ----> �
 8169 ----> �
  393 ----> ㅎ


## 3. sentencepiece
- 내부 단어 분리 알고리즘을 사용하기 위해서, 데이터에 단어 토큰화를 먼저 진행한 상태여야 한다면 이 단어 분리 알고리즘을 모든 언어에 사용하는 것은 쉽지 않습니다. 영어와 달리 한국어와 같은 언어는 단어 토큰화부터가 쉽지 않기 때문입니다. 그런데, 이런 사전 토큰화 작업(pretokenization)없이 전처리를 하지 않은 데이터(raw data)에 바로 단어 분리 토크나이저를 사용할 수 있다면, 이 토크나이저는 그 어떤 언어에도 적용할 수 있는 토크나이저가 될 것입니다. 센텐스피스는 이 이점을 살려서 구현되었습니다. 센텐스피스는 사전 토큰화 작업없이 단어 분리 토큰화를 수행하므로 언어에 종속되지 않습니다.
- Sentencepiece의 학습 데이터로는 빈 칸이 포함되지 않은 문서 집합이어야 합니다.

### 3-1. IMDB 리뷰

In [ ]:
# wget.download("https://github.com/euphoris/datasets/raw/master/imdb.zip")

100% [..............................................................................] 34005 / 34005

'imdb.zip'

In [ ]:
data = pd.read_csv("imdb.zip")

In [ ]:
data.head()

review  sentiment
0  A very, very, very slow-moving, aimless movie ...          0
1  Not sure who was more lost - the flat characte...          0
2  Attempting artiness with black & white and cle...          0
3         Very little music or anything to speak of.          0
4  The best scene in the movie was when Gerardo i...          1

In [ ]:
with open("review.txt", "w", encoding="utf8") as f:
    f.write("\n".join(df["review"]))

In [ ]:
sp.SentencePieceTrainer.Train("--input=review.txt --model_prefix=imdb --vocab_size=1000")

- `input` : 학습시킬 파일
- `model_prefix` : 만들어질 모델 이름
- `vocab_size` : 단어 집합의 크기
- `model_type` : 사용할 모델 (unigram(default), bpe, char, word)
- `max_sentence_length`: 문장의 최대 길이
- `pad_id`, `pad_piece`: pad token id, 값
- `unk_id`, `unk_piece`: unknown token id, 값
- `bos_id`, `bos_piece`: begin of sentence token id, 값
- `eos_id`, `eos_piece`: end of sequence token id, 값
- `user_defined_symbols`: 사용자 정의 토큰

In [ ]:
tokens = pd.read_csv("imdb.vocab", sep="\t", header=None, quoting=csv.QUOTE_NONE)

In [ ]:
tokens.sample(10)

0        1
52      an -5.77179
143     se -6.69717
130  ▁some -6.59937
508    ial -8.14767
302    ies -7.46211
16       a -4.47403
280      O -7.35558
299   ▁pro -7.45394
15       e -4.47045
56    ▁you -5.85125

In [ ]:
spp = sp.SentencePieceProcessor()
spp.load("imdb.model")

True

In [ ]:
text = df.loc[0, "review"]
print(text)
print(spp.encode_as_pieces(text))

A very, very, very slow-moving, aimless movie about a distressed, drifting young man.
['▁A', '▁very', ',', '▁very', ',', '▁very', '▁slow', '-', 'moving', ',', '▁a', 'im', 'less', '▁movie', '▁about', '▁a', '▁dist', 're', 's', 's', 'ed', ',', '▁dri', 'ft', 'ing', '▁you', 'ng', '▁man', '.']


### 3-2. 네이버 영화 리뷰

In [ ]:
# wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt

In [ ]:
ratings_df = pd.read_table("ratings.txt")
ratings_df = ratings_df.dropna(subset=["document"])

In [ ]:
ratings_df[:5]

id                                           document  label
0   8112052                                어릴때보고 지금다시봐도 재밌어요ㅋㅋ      1
1   8132799  디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...      1
2   4655635               폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.      1
3   9251303  와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...      1
4  10067386                        안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.      1

In [ ]:
with open("naver_review.txt", "w", encoding="utf8") as f:
    f.write("\n".join(ratings_df["document"]))

입력이 반드시 txt 파일이어야 하므로 txt 파일에 저장해줍니다.

In [ ]:
sp.SentencePieceTrainer.Train("--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe")

- `.model`, `.vocab` 파일 두개가 생성 됩니다.  
- `.vocab` 에서 학습된 subwords를 확인할 수 있습니다.

In [ ]:
subws = pd.read_csv("naver.vocab", sep="\t", header=None, quoting=csv.QUOTE_NONE)

In [ ]:
subws.sample(10)

0     1
4165      톤 -4162
3403      공 -3400
939      부분  -936
646     ▁어디  -643
707      ▁천  -704
2688     원이 -2685
3998      엘 -3995
1918  ▁좋았어요 -1915
527      스터  -524
181      ▁눈  -178

In [ ]:
spp = sp.SentencePieceProcessor()
spp.load("naver.model")

True

In [ ]:
print(spp.DecodePieces(['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']))
print(spp.IdToPiece(4))
print(spp.PieceToId("영화"))
print(spp.DecodeIds([54, 200, 821, 85]))

뭐 이딴 것도 영화냐.
영화
4
진짜 최고의 영화입니다 ᄏᄏ


enable_sampling=True 일 때 Drop-out이 적용되며 alpha=0.1은 10% 확률로 dropout 한다는 의미이다.

In [ ]:
for _ in range(5):
    print(spp.encode("진짜 최고의 영화입니다 ㅋㅋ", out_type=str, enable_sampling=True, alpha=0.5, nbest_size=-1))

['▁', '진', '짜', '▁최고의', '▁', '영화', '입니다', '▁', 'ᄏ', 'ᄏ']
['▁진짜', '▁', '최', '고', '의', '▁', '영화', '입', '니', '다', '▁', 'ᄏ', 'ᄏ']
['▁', '진짜', '▁', '최고', '의', '▁', '영화', '입', '니', '다', '▁ᄏᄏ']
['▁진', '짜', '▁', '최', '고', '의', '▁영화', '입니다', '▁', 'ᄏ', 'ᄏ']
['▁진짜', '▁', '최', '고', '의', '▁영화입니다', '▁', 'ᄏ', 'ᄏ']


In [ ]:
print(spp.encode("진짜 최고의 영화입니다 ㅋㅋ", out_type=str))
print(spp.encode("진짜 최고의 영화입니다 ㅋㅋ", out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]
